# 🔄 Transformação de Dados - Segurança Pública SP

Este notebook realiza a **transformação** dos dados brutos extraídos do portal da SSP-SP.

## Objetivos:
- Carregar CSVs brutos de `data/raw/`
- Padronizar nomes de colunas
- Limpar e tratar dados
- Criar coluna `taxa_crime_por_100k`
- Categorizar tipos de crime
- Salvar dados processados em `data/processed/`

---

## 1. Importar Bibliotecas e Funções de Transformação

In [ ]:
# Importar bibliotecas necessárias
import sys
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import logging

# Adicionar o diretório src ao path
sys.path.append(str(Path().resolve().parent / 'src'))

# Importar funções de transformação
from transform import (
    clean_column_names,
    remove_duplicates,
    handle_missing_values,
    normalize_dates,
    categorize_crimes,
    aggregate_by_region,
    calculate_crime_rate,
    validate_data
)

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Bibliotecas e funções de transformação importadas com sucesso!")

## 2. Configurar Diretórios e Caminhos

In [ ]:
# Definir diretórios do projeto
PROJECT_ROOT = Path().resolve().parent
DATA_RAW_DIR = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
DATA_INTERIM_DIR = PROJECT_ROOT / 'data' / 'interim'

# Criar diretórios se não existirem
DATA_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
DATA_INTERIM_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Diretório de dados brutos: {DATA_RAW_DIR}")
print(f"📁 Diretório de dados processados: {DATA_PROCESSED_DIR}")
print(f"📁 Diretório de dados intermediários: {DATA_INTERIM_DIR}")

## 3. Carregar Dados Brutos

In [ ]:
# Listar arquivos CSV disponíveis em data/raw
csv_files = list(DATA_RAW_DIR.glob('*.csv'))

print(f"📂 Arquivos CSV encontrados em data/raw/: {len(csv_files)}\n")
for i, file in enumerate(csv_files, 1):
    size_kb = file.stat().st_size / 1024
    print(f"  {i}. {file.name} ({size_kb:.2f} KB)")

# Carregar o arquivo de exemplo criado no notebook de extração
arquivo_exemplo = DATA_RAW_DIR / "exemplo_dados_ssp_2024.csv"

if arquivo_exemplo.exists():
    df_raw = pd.read_csv(arquivo_exemplo, sep=';', encoding='utf-8-sig')
    print(f"\n✅ Arquivo carregado: {arquivo_exemplo.name}")
    print(f"📊 Dimensões: {df_raw.shape[0]} linhas x {df_raw.shape[1]} colunas")
    print(f"\n📋 Primeiras 5 linhas:")
    display(df_raw.head())
else:
    print(f"\n⚠️ Arquivo de exemplo não encontrado!")
    print("Execute primeiro o notebook '1_extracao.ipynb'")

## 4. Etapa 1: Padronizar Nomes de Colunas

Usando a função `clean_column_names()` para:
- Converter para minúsculas
- Remover acentos
- Substituir espaços por underscores
- Remover caracteres especiais

In [ ]:
# Verificar nomes de colunas antes da transformação
print("📋 Colunas ANTES da padronização:")
print(df_raw.columns.tolist())

# Aplicar função de limpeza de nomes de colunas
df_transformed = clean_column_names(df_raw)

print("\n✅ Colunas APÓS a padronização:")
print(df_transformed.columns.tolist())

# Comparação lado a lado
print("\n📊 Comparação:")
for old, new in zip(df_raw.columns, df_transformed.columns):
    if old != new:
        print(f"  {old:<20} → {new}")
    else:
        print(f"  {old:<20} (sem alteração)")

print(f"\n✅ Padronização de nomes de colunas concluída!")
df_transformed.head()

## 5. Etapa 2: Remover Duplicatas

Usando a função `remove_duplicates()` para identificar e remover registros duplicados:

In [ ]:
# Verificar duplicatas antes da remoção
duplicatas_antes = df_transformed.duplicated().sum()
print(f"🔍 Registros duplicados encontrados: {duplicatas_antes}")

# Remover duplicatas
registros_antes = len(df_transformed)
df_transformed = remove_duplicates(df_transformed)
registros_depois = len(df_transformed)

print(f"\n📊 Registros ANTES: {registros_antes:,}")
print(f"📊 Registros DEPOIS: {registros_depois:,}")
print(f"🗑️  Registros removidos: {registros_antes - registros_depois:,}")

if registros_antes == registros_depois:
    print("\n✅ Nenhuma duplicata encontrada!")
else:
    print(f"\n✅ {registros_antes - registros_depois} duplicatas removidas com sucesso!")

## 6. Etapa 3: Tratar Valores Ausentes

Usando a função `handle_missing_values()` com diferentes estratégias:

In [ ]:
# Verificar valores ausentes
print("🔍 Valores ausentes por coluna:")
valores_nulos = df_transformed.isnull().sum()
print(valores_nulos)

total_nulos = valores_nulos.sum()
print(f"\n📊 Total de valores ausentes: {total_nulos}")

if total_nulos > 0:
    # Aplicar estratégia de preenchimento com zero (para dados de criminalidade)
    # Pode usar 'drop', 'fill_zero', 'fill_mean', 'fill_median'
    df_transformed = handle_missing_values(df_transformed, strategy='fill_zero')
    print("\n✅ Valores ausentes preenchidos com zero")
else:
    print("\n✅ Nenhum valor ausente encontrado!")

# Verificar novamente
print("\n🔍 Verificação pós-tratamento:")
print(df_transformed.isnull().sum())

## 7. Etapa 4: Categorizar Tipos de Crime

Usando a função `categorize_crimes()` para agrupar crimes em categorias:

In [ ]:
# Verificar tipos de crime únicos
print("🔍 Tipos de crime únicos no dataset:")
print(df_transformed['tipo_crime'].unique())
print(f"\nTotal de tipos diferentes: {df_transformed['tipo_crime'].nunique()}")

# Aplicar categorização
df_transformed = categorize_crimes(df_transformed, crime_column='tipo_crime')

# Verificar distribuição por categoria
print("\n📊 Distribuição por Categoria de Crime:")
distribuicao = df_transformed.groupby('categoria_crime').agg({
    'ocorrencias': 'sum',
    'vitimas': 'sum'
}).sort_values('ocorrencias', ascending=False)

print(distribuicao)

# Visualizar
print(f"\n✅ Nova coluna 'categoria_crime' criada!")
print(f"\n📋 Amostra dos dados:")
df_transformed[['tipo_crime', 'categoria_crime', 'ocorrencias']].head(10)

## 8. Etapa 5: Criar Coluna Taxa de Crime por 100k Habitantes

Usando a função `calculate_crime_rate()` para calcular a taxa de criminalidade:

In [ ]:
# Criar dataset de população (dados aproximados para demonstração)
# Em produção, você carregaria dados reais do IBGE ou outra fonte oficial

populacao_sp = {
    'municipio': ['São Paulo', 'Campinas', 'Santos', 'Guarulhos', 'Ribeirão Preto'],
    'populacao': [12396372, 1223237, 433153, 1291771, 711825]  # Dados aproximados
}

df_populacao = pd.DataFrame(populacao_sp)

print("📊 Dados de População:")
print(df_populacao)
print()

# Verificar municípios antes do merge
print(f"🔍 Municípios no dataset de criminalidade: {df_transformed['municipio'].unique()}")
print(f"🔍 Municípios no dataset de população: {df_populacao['municipio'].unique()}")

# Aplicar cálculo de taxa de criminalidade
df_transformed = calculate_crime_rate(df_transformed, df_populacao)

print("\n✅ Coluna 'taxa_criminalidade' criada!")
print(f"\n📋 Amostra com as novas colunas:")
df_transformed[['municipio', 'tipo_crime', 'ocorrencias', 'populacao', 'taxa_criminalidade']].head(10)

## 9. Validação dos Dados Transformados

Usando a função `validate_data()` para verificar a integridade:

In [ ]:
# Validar dados transformados
validation_results = validate_data(df_transformed)

print("="*60)
print("📊 RELATÓRIO DE VALIDAÇÃO DOS DADOS TRANSFORMADOS")
print("="*60)

print(f"\n1️⃣ Total de Registros: {validation_results['total_registros']:,}")

print(f"\n2️⃣ Total de Colunas: {len(validation_results['colunas'])}")
print("   Colunas:", ", ".join(validation_results['colunas']))

print(f"\n3️⃣ Valores Nulos:")
if sum(validation_results['valores_nulos'].values()) == 0:
    print("   ✅ Nenhum valor nulo encontrado!")
else:
    for col, count in validation_results['valores_nulos'].items():
        if count > 0:
            print(f"   - {col}: {count}")

print(f"\n4️⃣ Duplicatas: {validation_results['duplicatas']}")

print(f"\n5️⃣ Estatísticas Descritivas:")
print(df_transformed[['ocorrencias', 'vitimas', 'taxa_criminalidade']].describe())

print("\n" + "="*60)

## 10. Visualização dos Dados Transformados

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

# Criar visualizações
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Gráfico 1: Taxa de Criminalidade por Município
taxa_por_municipio = df_transformed.groupby('municipio')['taxa_criminalidade'].mean().sort_values(ascending=True)
taxa_por_municipio.plot(kind='barh', ax=axes[0, 0], color='darkred')
axes[0, 0].set_title('Taxa de Criminalidade Média por Município (por 100k hab)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Taxa por 100k habitantes', fontsize=11)
axes[0, 0].set_ylabel('Município', fontsize=11)

# Gráfico 2: Distribuição por Categoria de Crime
categoria_crimes = df_transformed.groupby('categoria_crime')['ocorrencias'].sum().sort_values(ascending=False)
colors = ['#e74c3c', '#3498db', '#f39c12', '#2ecc71']
axes[0, 1].pie(categoria_crimes, labels=categoria_crimes.index, autopct='%1.1f%%', 
               colors=colors, startangle=90)
axes[0, 1].set_title('Distribuição por Categoria de Crime', fontsize=14, fontweight='bold')

# Gráfico 3: Top 10 Tipos de Crime
top_crimes = df_transformed.groupby('tipo_crime')['ocorrencias'].sum().sort_values(ascending=False).head(10)
top_crimes.plot(kind='bar', ax=axes[1, 0], color='steelblue')
axes[1, 0].set_title('Top 10 Tipos de Crime (Total de Ocorrências)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Tipo de Crime', fontsize=11)
axes[1, 0].set_ylabel('Número de Ocorrências', fontsize=11)
axes[1, 0].tick_params(axis='x', rotation=45)

# Gráfico 4: Evolução Mensal
evolucao_mensal = df_transformed.groupby('mes')['ocorrencias'].sum()
axes[1, 1].plot(evolucao_mensal.index, evolucao_mensal.values, marker='o', 
                linewidth=2.5, markersize=8, color='darkgreen')
axes[1, 1].fill_between(evolucao_mensal.index, evolucao_mensal.values, alpha=0.3, color='lightgreen')
axes[1, 1].set_title('Evolução Mensal de Ocorrências', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Mês', fontsize=11)
axes[1, 1].set_ylabel('Número de Ocorrências', fontsize=11)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xticks(range(1, 11))

plt.tight_layout()
plt.show()

print("✅ Visualizações criadas com sucesso!")

## 11. Salvar Dados Processados em `data/processed/`

Salvando os dados transformados em múltiplos formatos:

In [ ]:
# Definir nome do arquivo de saída
timestamp = datetime.now().strftime('%Y%m%d')
output_filename = f"dados_processados_ssp_{timestamp}"

# 1. Salvar em CSV
csv_path = DATA_PROCESSED_DIR / f"{output_filename}.csv"
df_transformed.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"✅ CSV salvo: {csv_path}")
print(f"   Tamanho: {csv_path.stat().st_size / 1024:.2f} KB")

# 2. Salvar em Parquet (mais eficiente)
parquet_path = DATA_PROCESSED_DIR / f"{output_filename}.parquet"
df_transformed.to_parquet(parquet_path, index=False)
print(f"\n✅ Parquet salvo: {parquet_path}")
print(f"   Tamanho: {parquet_path.stat().st_size / 1024:.2f} KB")

# 3. Salvar versão agregada por município
df_agregado = aggregate_by_region(df_transformed, region_col='municipio')
agregado_path = DATA_PROCESSED_DIR / f"dados_agregados_municipio_{timestamp}.csv"
df_agregado.to_csv(agregado_path, index=False, encoding='utf-8-sig')
print(f"\n✅ Dados agregados salvos: {agregado_path}")
print(f"   Tamanho: {agregado_path.stat().st_size / 1024:.2f} KB")

print(f"\n📊 Total de registros processados: {len(df_transformed):,}")
print(f"📊 Total de colunas: {len(df_transformed.columns)}")

## 12. Salvar Metadados da Transformação

In [ ]:
import json

# Criar metadados da transformação
metadata_transformacao = {
    'data_processamento': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'notebook': '2_transformacao.ipynb',
    'arquivo_origem': 'exemplo_dados_ssp_2024.csv',
    'arquivos_saida': [
        f"{output_filename}.csv",
        f"{output_filename}.parquet",
        f"dados_agregados_municipio_{timestamp}.csv"
    ],
    'transformacoes_aplicadas': [
        'Padronização de nomes de colunas',
        'Remoção de duplicatas',
        'Tratamento de valores ausentes (fill_zero)',
        'Categorização de tipos de crime',
        'Cálculo de taxa de criminalidade por 100k habitantes',
        'Agregação por município'
    ],
    'dados_processados': {
        'total_registros': int(len(df_transformed)),
        'total_colunas': int(len(df_transformed.columns)),
        'colunas': list(df_transformed.columns),
        'periodo': {
            'ano': int(df_transformed['ano'].iloc[0]),
            'mes_inicio': int(df_transformed['mes'].min()),
            'mes_fim': int(df_transformed['mes'].max())
        }
    },
    'qualidade': {
        'valores_nulos': int(df_transformed.isnull().sum().sum()),
        'duplicatas': int(df_transformed.duplicated().sum()),
        'registros_validos': int(len(df_transformed))
    },
    'estatisticas': {
        'total_ocorrencias': int(df_transformed['ocorrencias'].sum()),
        'total_vitimas': int(df_transformed['vitimas'].sum()),
        'taxa_media_criminalidade': float(df_transformed['taxa_criminalidade'].mean()),
        'municipios_analisados': int(df_transformed['municipio'].nunique()),
        'tipos_crime': int(df_transformed['tipo_crime'].nunique())
    }
}

# Salvar metadados
metadata_path = DATA_PROCESSED_DIR / f'metadata_transformacao_{timestamp}.json'
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata_transformacao, f, indent=2, ensure_ascii=False)

print(f"✅ Metadados salvos em: {metadata_path}")
print(f"\n📋 Resumo da Transformação:")
print(json.dumps(metadata_transformacao, indent=2, ensure_ascii=False))

## 13. Resumo e Próximos Passos

### ✅ Resumo da Transformação

Neste notebook, realizamos as seguintes transformações:

1. ✅ **Padronização de Nomes de Colunas** - Converteu para minúsculas, removeu acentos e caracteres especiais
2. ✅ **Remoção de Duplicatas** - Identificou e removeu registros duplicados
3. ✅ **Tratamento de Valores Ausentes** - Preencheu valores nulos com zeros
4. ✅ **Categorização de Crimes** - Agrupou tipos de crime em categorias (Violentos, Patrimoniais, Trânsito, Outros)
5. ✅ **Cálculo da Taxa de Criminalidade** - Criou coluna `taxa_criminalidade` por 100k habitantes
6. ✅ **Agregação por Município** - Gerou dataset resumido por região
7. ✅ **Validação de Dados** - Verificou integridade e qualidade dos dados
8. ✅ **Visualizações** - Criou gráficos para análise exploratória
9. ✅ **Salvamento de Dados** - Exportou para CSV e Parquet em `data/processed/`
10. ✅ **Metadados** - Registrou informações sobre o processo de transformação

### 📊 Resultados Obtidos

- **Dados limpos e padronizados**
- **Taxa de criminalidade por 100k habitantes calculada**
- **Categorização de crimes aplicada**
- **Dados prontos para análise e carga**

### 📝 Próximos Passos

1. **Carga (Load)**: Execute o notebook `3_carga.ipynb` para carregar os dados em um banco de dados
2. **Análise**: Use o notebook `4_analise_dados.ipynb` para análises exploratórias avançadas
3. **Visualizações**: Crie dashboards e relatórios com os dados processados

### 📂 Arquivos Gerados

- `data/processed/dados_processados_ssp_YYYYMMDD.csv` - Dados completos em CSV
- `data/processed/dados_processados_ssp_YYYYMMDD.parquet` - Dados em formato Parquet
- `data/processed/dados_agregados_municipio_YYYYMMDD.csv` - Dados agregados por município
- `data/processed/metadata_transformacao_YYYYMMDD.json` - Metadados do processo

In [ ]:
# Verificar todos os arquivos criados
print("="*60)
print("📂 ARQUIVOS CRIADOS NA TRANSFORMAÇÃO")
print("="*60)

print(f"\n📁 Diretório: {DATA_PROCESSED_DIR}\n")
for file in sorted(DATA_PROCESSED_DIR.glob('*')):
    size_kb = file.stat().st_size / 1024
    print(f"  📄 {file.name:<50} ({size_kb:,.2f} KB)")

print("\n" + "="*60)
print("✅ TRANSFORMAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*60)
print("\n🎯 Dados transformados e prontos para:")
print("   1. Carga em banco de dados (notebook 3_carga.ipynb)")
print("   2. Análise exploratória avançada (notebook 4_analise_dados.ipynb)")
print("   3. Criação de dashboards e visualizações")
print("\n🚀 Próximo passo: Execute o notebook '3_carga.ipynb'")